In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cvxpy as cp

In [2]:
# For data preprocess import
import re
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk

In [3]:
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
from string import punctuation
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions

In [5]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [WinError 10065] A
[nltk_data]     socket operation was attempted to an unreachable host>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


False

In [6]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

%matplotlib inline

# DATA IMPORT

In [8]:
import pandas as pd

tweets = pd.read_csv(r'D:\Users\softphea\Downloads\BUSINESS CASE_GROUP PROJECT\DATASET\Dataset_tiktok shop.csv')
print(tweets)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 13762: invalid start byte

In [ ]:
tweets  

In [ ]:
tweets.head()

In [ ]:
tweets_df = tweets[['full_text']]
tweets_df.head()

In [ ]:
tweets_df.shape

# DATA PREPROCESSING

In [ ]:
tweets.drop_duplicates(inplace = True, subset="full_text")
tweets.duplicated()



In [ ]:
tweets_df

In [ ]:
#To view the information of data
tweets_df.info()

In [ ]:
#To create dataframe
cleaned = tweets_df[["full_text"]]

#Rearrange the index
cleaned_df = cleaned.reset_index(drop = True)
cleaned_df

In [ ]:
#To remove NaN
cleaned_df = cleaned_df.dropna()

#To view the data after removing NaN
cleaned_df

In [ ]:
#Lowercase coversion
cleaned_df['lowercase'] = cleaned_df.full_text.str.lower()
cleaned_df.head()

In [ ]:
#to convert the datatype to string
cleaned_df.full_text = cleaned_df.full_text.astype(str)

In [ ]:
#Remove URL link
import re
cleaned_df['remove_link'] = cleaned_df.lowercase.apply(lambda x: re.sub(r'https?:\/\/\S+', '',x))
cleaned_df.lowercase.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
                                                                                                                                              
cleaned_df.head()

In [ ]:
#remove twitter handles (mention)
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
        return input_txt

In [ ]:
#remove twitter handles (user)
cleaned_df['mention'] = np.vectorize(remove_pattern)(cleaned_df['remove_link'], "@[\w]*")

cleaned_df.head()

In [ ]:
import string
dir(string)

In [ ]:
string.punctuation

In [ ]:
for char in cleaned_df:
    if char in string.punctuation:
        cleaned_df = cleaned_df.replace(char, '')
        print(cleaned_df)

In [ ]:
def clean_text(full_text):
    for char in full_text:
        full_text = full_text.replace(char, '')
        return full_text

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df['remove_link'].apply(lambda x: clean_text(x))

In [ ]:
#Remove punctuation
cleaned_df['punctuation'] = cleaned_df.remove_link.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))

cleaned_df.head()

In [ ]:
cleaned_df['expand_word'] = cleaned_df['punctuation'].apply(lambda x: [contractions.fix(word) for word in x.split()])

In [ ]:
cleaned_df['expand_word'] = [' '.join(map(str, l)) for l in cleaned_df['expand_word']]

cleaned_df.head()

In [ ]:
#Remove #
cleaned_df['hashtag']= cleaned_df.expand_word.apply(lambda x: re.sub("#[A-Za-z0-9_]+","", x))

cleaned_df.head(3)

In [ ]:
#remove short word that has length word 2 or less
cleaned_df['remove_short'] = cleaned_df['hashtag'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

cleaned_df.head()

In [ ]:
#tokenization
import nltk
nltk.download('punkt_tab')

cleaned_df['tokenized'] = cleaned_df['remove_short'].apply(word_tokenize)
cleaned_df.head()

In [ ]:
#Stop word library
nltk.download("stopwords")
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#Stopword removal

import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))

#set of stop words
stop_words = set(stopwords.words('english'))

cleaned_df['stopwords_removed'] = cleaned_df.tokenized.apply(lambda x: [word for word in x if word not in stop_words])

In [ ]:
cleaned_df.head()

In [ ]:
#pos tag to label the type of word
import nltk
nltk.download('averaged_perceptron_tagger_eng')

cleaned_df['pos_tags'] = cleaned_df['stopwords_removed'].apply(nltk.tag.pos_tag)
cleaned_df.head()

In [ ]:
#To convert pos tag to word net to use lemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
cleaned_df['wordnet_pos'] = cleaned_df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x]) 

In [ ]:
cleaned_df.head()

In [ ]:
#Lemmatizer
wnl = WordNetLemmatizer()
cleaned_df['lemmatized'] = cleaned_df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
cleaned_df.head()

In [ ]:
#Saving dataframe in csv
cleaned_df.to_csv('Preprocessed_Data')
# tweets_df.to_excel("preprocessed_data.xlsx", index=False)